# Creacion de SQL script

Este codigo tiene le objetivo de tomar los datos del json original, procesarlo y pasarlo a un script SQL para generar el esquema de tablas
1. obtencion de los JSON
2. definicion de esquema de BBDD
3. filtrado de pacientes, solo tomaremos los paciences contenidos en la tabla evolucion
4. modificacion de datos de pacientes
5. almacenar los datos de evoluciones y pacientes en archivos

## 1. Obtencion de los JSON

In [22]:
import os
import pandas as pd
from dotenv import load_dotenv
import json

load_dotenv()

# obtener pacientes
pacientes_json = []
with open("./json/pacientes.json", "r") as f:
    pacientes_json = json.load(f)
pacientes = pacientes_json[2]["data"]
    
# obtener evoluciones
evoluciones_json = []
with open("./json/evolucion.json", "r") as f:
    evoluciones_json = json.load(f)
evoluciones = evoluciones_json[2]["data"]


## 2. Definicion de esquema de BBDD

In [ ]:
CREATE TABLE evolucion (
    id INTEGER PRIMARY KEY,
    id_paciente INTEGER,
    fecha DATE,
    edad INTEGER,
    uni_edad VARCHAR(20),
    edad_anios FLOAT,
    edad_texto VARCHAR(50),
    peso FLOAT,
    talla FLOAT,
    imc FLOAT,
    pc FLOAT,
    motivo VARCHAR(255),
    conducta TEXT,
    FOREIGN KEY (id_paciente) REFERENCES paciente(id_paciente)
);

In [ ]:
CREATE TABLE paciente (
    id_paciente INTEGER PRIMARY KEY,
    apellido VARCHAR(100),
    nombre VARCHAR(100),
    fecha_nac DATE,
    sexo VARCHAR(10),
    edad VARCHAR(20),
    dni VARCHAR(20),
    localidad VARCHAR(100),
    obra_social VARCHAR(100),
    afiliado_nro VARCHAR(50),
    telefono VARCHAR(100),
    telefono_numero VARCHAR(20),
    email VARCHAR(100),
    especialidad VARCHAR(100),
    diagnostico VARCHAR(255),
    enfermedad_base VARCHAR(255),
    ant_perinatales TEXT,
    ant_familiares TEXT,
    registro VARCHAR(50),
    fecha_reg VARCHAR(50)
);

## 3. filtrado de pacientes

Para hacer esto debenmos tomar solo los pacientes que tienen consultas

In [23]:
# iterar sobre las consultas
ids_pacientes_con_evoluciones = set()
for evolucion in evoluciones:
    id_paciente = evolucion["id_paciente"]
    if id_paciente not in ids_pacientes_con_evoluciones:
        ids_pacientes_con_evoluciones.add(id_paciente)
# filtrar pacientes
pacientes_con_evoluciones = [p for p in pacientes if p["id_paciente"] in ids_pacientes_con_evoluciones] 

# crear dataframes
df_pacientes = pd.DataFrame(pacientes_con_evoluciones)
df_evoluciones = pd.DataFrame(evoluciones)

print(f"Pacientes original cantidad: {pacientes.__len__()}")
print(f"Pacientes con evoluciones cantidad: {pacientes_con_evoluciones.__len__()}")
print(f"Evoluciones cantidad: {evoluciones.__len__()}")


Pacientes original cantidad: 315
Pacientes con evoluciones cantidad: 70
Evoluciones cantidad: 79


# 4. Modificar los datos personales reales
Modificar los datos pidiendo al LLM que genere datos ficticios
- apellido
- nombre
- dni
- afiliado_nro
- telefono
- telefono_numero
- email

In [ ]:
## 1. instalar librerias
## 2. importar librerias
## 3. iterar sobre pacientes
## 4. llamar a la api para modificar los datos personales
## 5. crear dataframes


## 5. Persistir los datos en archivos 
En este punto debemos guardar los dataframes para subirlos a la BBDD

In [ ]:
## almacenar PACIENTES y EVOLUCIONES en archivos CSV o similares